In [1]:
import pandas as pd
import cv2 #bib opencv open sourve computer vision
import numpy as np
import os
from firebase.firebase import FirebaseApplication
firebase=FirebaseApplication('https://tes-ai-d3b26-default-rtdb.firebaseio.com/',None)

In [ ]:

from keras.preprocessing.image import img_to_array #Converts a PIL Image instance to a Numpy array.
import imutils #make basic image processing functions such as translation, rotation, resizing, skeletonization isplaying Matplotlib images, sorting contours, detecting edges, and much more easier with OpenCV 
import cv2  #openCV 
from keras.models import load_model #loding the train model from keras
import numpy as np

# parameters for loading data and images
detection_model_path = 'haarcascade_files/haarcascade_frontalface_default.xml' 
face_detection = cv2.CascadeClassifier(detection_model_path)
#choose the detection model from HAARcascade(frontal face)


emotion_model_path = 'models/_mini_XCEPTION.102-0.66.hdf5' #cnn simple Xception from Github

# hyper-parameters for bounding boxes shape
# loading models 
# to load a xml classifier
emotion_classifier = load_model(emotion_model_path, compile=False) #emotion classifier with mini-Xception

EMOTIONS = ["angry" ,"disgust","scared", "happy", "sad", "surprised", "neutral"] #number of classes 7

feelings_faces = [] #empty matrix
for index, emotion in enumerate(EMOTIONS):
    feelings_faces.append(cv2.imread('emojis/' + emotion + '.png', -1)) 

# starting video streaming
cv2.namedWindow('Your_face')
camera = cv2.VideoCapture(0)
while True:
    frame = camera.read()[1]
    #reading the frame
    frame = imutils.resize(frame,width=300) #resizing basic image processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #convert to gray level
    #to perform the detection:
    faces = face_detection.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=5,minSize=(30,30),flags=cv2.CASCADE_SCALE_IMAGE) #multi-échelle
    #cv.CascadeClassifier.detectMultiScale(image[, scaleFactor[, minNeighbors[, flags[, minSize[, maxSize]]]]]
    canvas = np.zeros((250, 300, 3), dtype="uint8")  #matrix 3D of zeros 
    frameClone = frame.copy() #make frame copy
    if len(faces) > 0: #faces multiscale 
        faces = sorted(faces, reverse=True, key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces 
                    # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
            # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0 #normalisation
        roi = img_to_array(roi) #convert image to an array
        roi = np.expand_dims(roi, axis=0) 
        
        preds = emotion_classifier.predict(roi)[0] #prediction of emotions  
        emotion_probability = np.max(preds) #max prediction as probability 
        label = EMOTIONS[preds.argmax()] #labels of 7 emotions 
    else: continue

 
    for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, preds)):
                # construct the label text
                text = "{}: {:.2f}%".format(emotion, prob * 100)

                # draw the label + probability bar on the canvas
               # emoji_face = feelings_faces[np.argmax(preds)]

                
                w = int(prob * 300) 
                cv2.rectangle(canvas, (7, (i * 35) + 5), (w, (i * 35) + 35), (0, 0, 255), -1)
                cv2.putText(canvas, text, (10, (i * 35) + 23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 2)
                cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
                cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH), (0, 0, 255), 2)
                firebase.put("acces",emotion,prob*100)
               # firebase.put("emo",'EMOTIONS',prob*100)
                
               # firebase.put("emo",'preds',prob)
                #firebase.put('Emotions',emotion,preds)
    #for c in range(0, 3):
        #frame[200:320, 10:130, c] = emoji_face[:, :, c] * \(emoji_face[:, :, 3] / 255.0) + frame[200:320,10:130, c] * (1.0 - emoji_face[:, :, 3] / 255.0)

   
    cv2.imshow('your_face',frameClone)
    cv2.imshow("Probabilities", canvas)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release() #open camera
cv2.destroyAllWindows() #


